# Real-Time Ecommerce Data Analytics

### Online Book Store

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [15]:
BASE_URL = "https://books.toscrape.com/"
HEADERS = {
    "User-Agent": "Mozilla/5.0"
}

In [16]:
def convert_rating(rating_class):
    rating_map = {
        "One": 1,
        "Two": 2,
        "Three": 3,
        "Four": 4,
        "Five": 5
    }
    for key in rating_map:
        if key in rating_class:
            return rating_map[key]
    return None

In [20]:
def scrape_books():
    books = []
    page_url = "catalogue/page-1.html"

    while page_url:
        url = BASE_URL + page_url
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.text, "html.parser")

        articles = soup.select("article.product_pod")

        for book in articles:
            title = book.h3.a["title"]
            price_text = book.select_one("p.price_color").text
            price = float("".join(ch for ch in price_text if ch.isdigit() or ch == "."))
            rating_class = book.select_one("p.star-rating")["class"]
            rating = convert_rating(rating_class)
            availability = book.select_one("p.instock.availability").text.strip()

            books.append({
                "Book Title": title,
                "Price (£)": float(price),
                "Rating": rating,
                "Availability": availability
            })

        next_btn = soup.select_one("li.next a")
        page_url = next_btn["href"] if next_btn else None
        time.sleep(1)

    return pd.DataFrame(books)

In [21]:
df = scrape_books()
df.head()

,Book Title,Price (£),Rating,Availability
0,A Light in the Attic,51.77,3,In stock
1,Tipping the Velvet,53.74,1,In stock
2,Soumission,50.10,1,In stock
3,Sharp Objects,47.82,4,In stock
4,Sapiens: A Brief History of Humankind,54.23,5,In stock


In [12]:
def clean_data(df):
    if df.empty:
        print("⚠ No data scraped. Skipping cleaning.")
        return df

    required_cols = {"Product Name", "Price (INR)", "Rating"}
    if not required_cols.issubset(df.columns):
        print("⚠ Expected columns missing:", df.columns.tolist())
        return df

    df = df.drop_duplicates()
    df = df[df["Price (INR)"] > 0]
    df["Rating"] = df["Rating"].fillna(df["Rating"].mean())
    df.reset_index(drop=True, inplace=True)

    return df

In [22]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Book Title    20 non-null     object 
 1   Price (£)     20 non-null     float64
 2   Rating        20 non-null     int64  
 3   Availability  20 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 772.0+ bytes


,Price (£),Rating
count,20.000000,20.000000
mean,38.048500,2.850000
std,15.135231,1.565248
min,13.990000,1.000000
25%,22.637500,1.000000
50%,41.380000,3.000000
75%,51.865000,4.000000
max,57.250000,5.000000


In [23]:
def clean_data(df):
    df = df.drop_duplicates()
    df["Availability"] = df["Availability"].str.replace("\n", "").str.strip()
    df.reset_index(drop=True, inplace=True)
    return df

clean_df = clean_data(df)
clean_df.head()

,Book Title,Price (£),Rating,Availability
0,A Light in the Attic,51.77,3,In stock
1,Tipping the Velvet,53.74,1,In stock
2,Soumission,50.10,1,In stock
3,Sharp Objects,47.82,4,In stock
4,Sapiens: A Brief History of Humankind,54.23,5,In stock


In [24]:
# Average price by rating
clean_df.groupby("Rating")["Price (£)"].mean()

Rating
1    40.018333
2    36.830000
3    42.316667
4    31.105000
5    39.750000
Name: Price (£), dtype: float64

In [25]:
# Count of books per rating
clean_df["Rating"].value_counts().sort_index()

Rating
1    6
2    3
3    3
4    4
5    4
Name: count, dtype: int64

In [26]:
clean_df.to_excel("books_to_scrape_analytics.xlsx", index=False)
print("✅ Excel file exported successfully")

✅ Excel file exported successfully
